# 번역기를 만들어보자

## 기계번역 이야기

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-15-1.machine_translation.max-800x600.png)

1940년대부터 1980년대까지의 기계번역은 **규칙 기반의 기계 번역(Rule-based Machine Translation)**이 주를 이루었습니다. 예를 들어, 2차 세계 대전 시절 미국이 러시아어를 영어로 번역하고자 했을 때, 러시아 문장에서 단어 하나 하나의 의미를 분석하고 그 단어들이 어떻게 문법적으로 연결되었는지를 분석하고 그 다음 구조 상의 의미를 분석한 뒤에 그제서야 영어와 의미적으로 맞게 단어들을 분리하고 마지막으로 영어 문법에 맞게 순서를 맞추고.. 꽤 복잡한 과정을 거쳤었죠.

![Img](https://aiffelstaticprd.blob.core.windows.net/media/original_images/E-15-2.ibm_paper.png)

1980년대 후반에 들어서 IBM에서 규칙 기반에서 벗어나려는 시도가 있었습니다. 언어학 기반의 규칙이 아닌 통계로 구현하려는 시도였는데요, 이를 **통계적 기걔 번역(Statistical Machine Translation)**이라고 합니다. 1980년대 당시 규칙 기반의 접근이 얼마나 답답했는지 그 팀의 리더 프레데리 제리닉은 이 당시 "매번 내가 언어학자를 해고할 때 마다, 언어 인식기의 성능이 올라갔다"라는 말을 남겼다는 소문이 있죠. 이 방법은 2010년 중반까지 주요한 접근으로 사용되었습니다.

통계기반 번역에 여러 머신러닝이 사용되었지만 딥러닝은 아니었습니다. 몇 차례 인공 신경망으로 번역을 제안하는 논문은 있었지만, 훈련 데이터도 작고 신경망의 크기도 작아서 주목받지 못했습니다. 그러나 2010년 중반에 들어 알고리즘과 하드웨어의 발전으로 딥러닝이 빛을 발하면서 달라졌습니다.

구글은 2016년 9월, 자신들의 구글 번역기에 **신경망 기계 번역(Neural Machine Translation, NMT)**을 도입하면서 획기적인 성능 개선을 이루었다고 발표했습니다. 통계기반 번역에서 신경망 기계번역으로 변경되면서 한층 더 높은 수준의 번역 능력을 가질 수 있게 되었는데요, 이때 사용된 신경망이 **seq2seq**입니다. 이번 시간에는 신경망 seq2seq를 이용하여 기계번역기를 만들어봅시다.

## 시퀀스를 처리하는 RNN

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-15-3.rnn_effectiveness.max-800x600.jpg)

앞서 시쿠너스 데이터를 처리하는 RNN에 대해서 배웠습니다. RNN은 어떻게 활용하느냐에 따라서 다양한 어플리케이션에서 사용할 수 있습니다.

위 그림은 다양한 RNN의 방법을 보여주고 있습니다. 기계 번역기 구현에서는 기본적으로 4번을 활용해야 합니다. 그 이유는 *'사람 말은 끝까지 들어봐야 한다'*는 말과 연관이 있는데요, 번역 또는 통역이라고 하는 것은 전체 문장을 모두 읽거나 듣고나서야 할 수 있기 때문입니다. 번역이나 통역에 5번을 사용하면 아직 사람 말도 다 하지 않았는데 단어 하나를 들을 때마다 그 순간을 번역해야 하는데, 그럼 문맥에 맞는 말이 나올 가능성이 적어지겠죠?

## seq2seq 

### seq2seq의 인코더-디코더 구조

앞서 번역기는 다양한 RNN의 유형 그림중에서도 4번을 사용한다고 했습니다. 사실 여기에서 조금 더 깊게 살펴보자면 약간 다릅니다. 아래 그림은 번역기의 기본 구조인 seq2seq 입니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-15-4.seq2seq.max-800x600.jpg)

seq2seq는 두개의 RNN 아키텍처를 연결한 구조입니다. 입력 문장을 받는 RNN을 **인코더**라고 하고, 두번째 RNN을 **디코더**라고 합니다.

아래 그림에서 Encoder는 Feature Extractor의 역할을 합니다. 어떤 데이터 $X$를 해석하기 위한 저차원의 feature vector $z$를 만들어냅니다. 반면에 Decoder는 저차원의 Feature $z$로부터 정보를 복원해서 다시 어떤 데이터 $X'$를 재생성하는 역할을 합니다.

![IMG](https://aiffelstaticprd.blob.core.windows.net/media/images/E-15-5.encdec.max-800x600.png)

우리가 오늘 살펴볼 seq2seq 모델은 위 인코더-디코더 모델에서 인코더와 디코더 모델이 모두 RNN인 경우라고 볼 수 있을 것입니다. 그렇다면 seq2seq의 feature vector는 무엇일까요? 바로 인코더 RNN이 입력 문장을 해석해서 만들어 낸 hidden state 벡터일 것입니다. 즉, A언어의 문장 X를 z라는 hidden state로 해석한 후 z를 다시 B 언어의  문장 Y로 재생성 하는 것입니다. 그러므로 인코더에서 필요한 것은 *마지막 time step의 hidden state*입니다. 그리고 이를 두번째 RNN인 디코더에 전달합니다.

디코더는 인코더의 마지막 time step의 hidden state를 전달받아 자신의 초기 Hidden state로 하고, 출력 문장을 생성해내기 시작합니다. 여기서는 특수 문자를 사용해서 출력 문장의 시작과 종료를 알려주어야 합니다. 위의 그림에서는 `_GO`와 `EOS`가 각각 시작 문자와 종료 문자에 해당합니다. 문헌에 따라서는 `SOS`와 `EOS`라고도 하는데, `SOS`는 start of sequence를 의미하며, `EOS`는 end of sequence를 의미합니다.

### Conditional Language Model

문장 생성기(Text Generator) 모델을 만들어 보았다면, 그러한 문장 생성기는 **언어 모델(Language Model)**을 구현한 것이라는 것을 알고 있을 것입니다.

언어 모델이란 n-1개의 단어 시퀀스 $w_1, \cdots, w_{n-1}$가 주어졌을 때, N번째 단어 $w_n$으로 무엇이 올지를 예측하는 확률 모델입니다. 파라미터 $\theta$로 모델링하는 언어 모델을 다음과 같이 표현할 수 있습니다.

$$P(w_n | w_1, …, w_{n-1};\theta )$$

우리가 알고 있는 RNN 계열의 모델들은 이러한 언어 모델을 구현하기에 적함한 것들입니다.

그런데, 언어 모델에 기반한 문장 생성기가 가지고 있는 한 가지 문제점이 있습니다. 그것은 바로, *어떤 말을 만들고 싶을지를 제어할 수 없다*는 점입니다. RNN 모델이 확률적으로 그 다음 나올 단어들을 순차적으로 만들어 나가게 되는데, 그것을 상황에 맞게 제어할 수 있다면 아주 유용할 것입니다. 그래서 사람들은 위 언어 모델을 **조건적 언어 모델(Conditional Language Model)**의 개념을 생각하게 됩니다. 말하자면 아래와 같은 형태가 될 것입니다.

$$P(w_n | w_1, …, w_{n-1}, c;\theta )$$

이 식과 다르게 $c$라는 것이 하나 더 붙었지요? 이 $c$를 이용해 기계에게 '아무 문장이나 만들지 말고 c에 적합한 문장을 만들어'라고 주문하고 싶은 것입니다.

기계번역이야말로 가장 대표적인 Conditional Language Model의 사례가 될 것입니다. 'X라는 영어 문장을 Y라는 프랑스어 문장으로 번역해!'라는 것은 바꾸어 말하면, '프랑스어 문장 Y를 만들어봐, 단 그 문장은 영어로는 X라는 뜻이어야 해'라는 뜻이 됩니다. 그런데 이 조건을 어떻게 문장 생성기에 넣어줄까요? 그렇습니다. 이 문장 X를 해석해서 $c$로 만드는 인코더를 또 다른 RNN으로 만드는 것입니다. 그렇게 만든 $c$를 다시 문장생성기인 디코더 RNN에 입력으로 넣어주는 모델을 만들어 낸 것이 바로 오늘 다루게 될 seq2seq입니다.



## 교사 강요 (teacher forcing)

seq2seq는 훈련 과정과 테스트 과정에서의 동장 방식이 다르다는 특징이 있습니다. 이전 스텝의 그림을 보면 디코더 RNN은 이전 time step의 출력을 현재 time step의 입력으로 사용한다는 특징을 가지고 있습니다. 그런데 이는 테스트 과정에서의 이야기이고, 훈련 과정은 조금 다른 방식을 사용합니다. 그 이유는 훈련 과정에서 이전 time step이 잘못된 예측을 한다면 이를 입력으로 한 현재 time step의 예측도 잘못될 수 있기 때문입니다. 

![img](https://aiffelstaticprd.blob.core.windows.net/media/original_images/E-15-6.teacher_forcing.png)


훈련 과정에서는 실제 정답 시퀀스를 알고 있는 상황이므로 이전 Time step의 예측값을 현재 Time step의 입력으로 사용하는 것이 아니라 이전 time step의 실제 값으로 사용할 수 있습니다. 이 작업을 **교사 강요(teacher forcing)**이라고 합니다. 이 기법은 seq2seq 뿐만 아니라 sequence 데이터의 생성모델에서 일반적으로 사용되는 기법이기도 합니다. 물론, 이는 모델이 훈련 데이터 외의 결과를 생성해내는 능력을 기르는 데에 조금 방해가 될 수 있다는 단점도 존재합니다.

## 단어 수준 VS 문자 수준

seq2seq **단어 수준(word level)** 또는 **문자 수준(character level)** 두 가지 방법으로 구현할 수 있습니다. 단어 수준이라고 함은 각 RNN의 time step의 입출력 단위가 단어 수준이라는 의미이고, 문자 수준이라 함은 RNN의 time step의 입출력 단위가 문자 수준, 영어에서는 알파벳 단위입니다.

실제 구현 자체는 문자 수준이 좀 더 쉬운데, 그 이유는 단어 수준 seq2seq의 경우 매 time step에서 디코더 RNN이 출력을 결정할 때, 훈련 데이터에 있는 전체 단어 집합의 수가 30만개라면 30만개 중 하나를 선택해야 하므로 출력층의 크기가 무려 30만이어야 합니다.

하지만 문자 수준으로 구현하면 영문자 알파벳은 26개에 불과하므로 출력층의 크기가 굉장히 작아집니다. 여기에 대, 소문자를 구분하지 않고 특수문자를 포함한다고 하더라도 출력층의 크기는 100을 넘지 않습니다.

그렇다면 단어 수준의 번역과 문자 수준의 번역 둘 중 어느 쪽이 더 유리할까요?

두 방법에는 장단점이 있습니다. 그리고 그 장단점은 서로 trade-off 관계이기도 합니다.

단어 수준의 번역을 위해서 사전을 구축한다고 생각해봅시다. 가장 극단적인 경우에 해당하는 한국어의 예를 들어봅시다.

> 먹다, 먹는다, 먹고, 먹을까,,

'먹다'라는 단어 하나에 이렇게 많은 변종이 있고, 이 의미가 전부 아주 약간 다릅니다. 이렇게 따지면 엄청나게 큰 단어 사전이 필요하게 됩니다. 영어에도 이런 문제가 있겠지만 한국어에 비하면 그 변화가 그리 심하지는 않습니다. 또 하나의 단어 수준의 접근의 어려움은 띄어쓰기의 문제입니다. 특히 띄어쓰기가 많이 생략되는 한국어, 일본어 같은 언어들에 있어 이러한 전처리가 큰 어려움의 원인이 됩니다.

그렇다면 문자 수준으로 번역하면 이런 문제가 없이 아주 깔끔하게 해결되겠죠? 그러나 단어를 문자 수준으로 쪼갠다는 것은 단어 안에 내재된 정보가 소실된다는 것을 의미합니다. 즉, 기계가 데이터를 통해 글자가 단어를 이루는 패턴까지 학습해야 한다는 것입니다. 그래서 충분한 데이터가 확보되지 않았다면 일반적으로 문자 수준의 번역이 단어 수준의 번역보다 품질이 떨어집니다.

하지만, 최신 자연어처리 흐름은 단어 수준이나 문자 수준의 번역이 아닌, 그 사이의 *subword 기반의 번역*이 주를 이루고 있습니다. 이에 대해서는 나중에 다루어봅시다.

이번 시간에는 전처리와 훈련 시간을 고려하여 문자 수준으로 구축해보겠습니다. 그리고 일단 문자 수준 번역기를 구축하고 나면 단어 수준 모델로 변환하는 것은 굉장히 쉽습니다. 이는 나중에 따로 연습해보세요!

---

## 데이터 전처리

이번 시간에 사용할 데이터는 <https://www.manythings.org/anki/> 에서 다운로드 하도록 합시다. 여기서는 프랑스어와 영어의 병렬 코퍼스인 `fra-eng.zip`을 다운받아 사용합니다. 아래 커맨드로 다운로드 받을 수 있습니다.

```bash
$ wget https://www.manythings.org/anki/fra-eng.zip
```

이 파일의 압축을 풀면 `fra.txt`라는 파일이 존재하는데 이 파일이 우리가 사용할 훈련 데이터입니다. 

In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

해당 파일을 데이터 프레임으로 읽어옵니다.

In [9]:
import os

file_path = os.path.dirname(os.path.abspath('__file__')) + r'/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print("전체 샘플의 수 : ", len(lines))
lines.sample(5)

전체 샘플의 수 :  178009


,eng,fra,cc
71861,She felt happy to see him.,Elle fut heureuse de le rencontrer.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
7585,Get off my car.,Descendez de ma voiture.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
19164,I felt very happy.,Je me sentais très heureux.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
41423,How could you do that?,Comment avez-vous pu faire ça ?,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
139026,I had nothing to do with that problem.,Je n'avais rien à voir avec ce problème.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...


세번째 열은 불필요하므로 제거합시다. 훈련 데이터는 5만개의 샘플로 줄이도록 하겠습니다.

In [10]:
lines = lines[['eng', 'fra']][:50000] # 5만개 샘플 사용
lines.sample(5)

,eng,fra
15793,It's a snowstorm.,C'est une tempête de neige.
39820,You should've phoned.,Tu aurais dû téléphoner.
37841,That narrows it down.,Ça diminue les possibilités.
34949,He's not in the mood.,Il n'est pas d'humeur.
9124,Stop grumbling.,Arrête de râler.


seq2seq 동작을 위해서 디코더의 입력과 예측에는 시작 토큰 `<sos>`와 종료 토큰 `<eos>`가 필요합니다. 이번에는 각각 `\t`와 `\n`을 사용하겠습니다.

In [11]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'

lines.fra = lines.fra.apply(lambda x : '\t ' + x + ' \n')
print("전체 샘플 수 : ", len(lines))
lines.sample(5)

전체 샘플 수 :  50000


,eng,fra
25717,My wife's pregnant.,\t Ma femme est enceinte. \n
13371,You can't leave.,\t Vous ne pouvez pas partir. \n
6352,"OK, I'm ready.","\t Bon, je suis prêt. \n"
3770,I need paint.,\t J'ai besoin de peinture. \n
40694,Don't be so impatient.,\t Ne soyez pas si impatientes ! \n


이제 단어장(vocabulary)을 만들고, 각 단어에 부여된 고유한 정수로 텍스트 시퀀스를 정수 시퀀스로 변환하는 정수 인코딩 과정을 거치겠습니다. 이때 영어와 프랑스어는 사용하는 언어가 다르므로 단어장을 별도로 만들어줍니다. 그리고 정상적으로 정수 시퀀스로 변환되었는지 3개의 행을 출력합니다.

In [12]:
eng_tokenizer = Tokenizer(char_level = True) # "문자" 단위로 Tokenizer를 생성합니다.
eng_tokenizer.fit_on_texts(lines.eng) # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng) # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[19, 3, 8], [10, 5, 8], [10, 5, 8]]

In [13]:
fra_tokenizer = Tokenizer(char_level = True) # 문자 단위로 Tokenizer 생성 (프랑스어)
fra_tokenizer.fit_on_texts(lines.fra) # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra) # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[11, 1, 19, 4, 1, 33, 1, 12],
 [11, 1, 3, 4, 13, 7, 5, 1, 33, 1, 12],
 [11, 1, 3, 4, 13, 7, 5, 14, 1, 12]]

단어장의 크기를 변수로 저장해줍니다. 0번 토큰을 고려하여 +1 하고 저장해줍니다.

In [15]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print("영어 단어징의 크기 :", eng_vocab_size)
print("프랑스어 단어장의 크기 :", fra_vocab_size)

영어 단어징의 크기 : 51
프랑스어 단어장의 크기 : 73


이제 영어 데이터와 프랑스어 데이터의 최대 길이를 각각 구해보겠습니다. 이는 패딩 (`<pad>`)을 위함입니다. 모델에 입력될 영어, 프랑스어 시퀀스의 길이가 일정해야 하므로, 최대 길이로 맞추고 남는 시퀀스 뒷부분을 패딩으로 채우게 됩니다.

In [16]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])

print("영어 시퀀스의 최대 길이 : ", max_eng_seq_len)
print("프랑스어 시퀀스의 최대 길이 : ", max_fra_seq_len)

영어 시퀀스의 최대 길이 :  23
프랑스어 시퀀스의 최대 길이 :  76


전체적인 통계 정보를 한꺼번에 출력해봅시다.

In [17]:
print("전체 샘플의 수 :", len(lines))
print("영어 단어장의 크기 :", eng_vocab_size)
print("프랑스어 단어장의 크기 :", fra_vocab_size)
print("영어 시퀀스의 최대 길이 :", max_eng_seq_len)
print("프랑스어 시퀀스의 최대 길이 :", max_fra_seq_len)

전체 샘플의 수 : 50000
영어 단어장의 크기 : 51
프랑스어 단어장의 크기 : 73
영어 시퀀스의 최대 길이 : 23
프랑스어 시퀀스의 최대 길이 : 76


인코더의 입력으로 사용되는 영어 시퀀스와 달리, 프랑스어 시퀀스는 2가지 버전으로 나누어 준비해야 합니다. 

하나는 디코더의 출력과 비교해야 할 정답 데이터로 사용해야 할 원래 목적에 따른 것입니다. 그리고 다른 하나는 이전 스텝에서 언급했던 **교사 강요**를 위해 디코더의 입력을 사용하기 위한 것입니다.

이때, 디코더의 입력으로 사용할 시퀀스는 `<eos>` 토큰이 필요 없고, 디코더의 출력과 비교할 시퀀스는 `<sos>`가 필요 없기 때문입니다. 가령, 영어로 'I am a person'이라는 문장을 프랑스어 'Je suis unepersonne'로 번역하는 번역기를 만든다고 해봅시다. 훈련 과정에서 디코더는 '< sos > Je suis une personne'를 입력받아서 'Je suis une personne < eos >'를 예측하도록 훈련되므로, 이런 방식으로 생성된 두가지 버전의 시퀀스를 준비해야합니다.

In [18]:
encoder_input = input_text

# 종료 토큰 제거
decoder_input = [[char for char in line if char != fra_tokenizer.word_index[eos_token]] for line in target_text]

# 시작 토큰 제거
decoder_target = [[char for char in line if char != fra_tokenizer.word_index[sos_token]] for line in target_text]

디코더의 입력과 출력을 각각 출력해봅시다.

In [19]:
print(decoder_input[:3])
print(decoder_target[:3])

[[11, 1, 19, 4, 1, 33, 1], [11, 1, 3, 4, 13, 7, 5, 1, 33, 1], [11, 1, 3, 4, 13, 7, 5, 14, 1]]
[[1, 19, 4, 1, 33, 1, 12], [1, 3, 4, 13, 7, 5, 1, 33, 1, 12], [1, 3, 4, 13, 7, 5, 14, 1, 12]]


디코더의 입력의 경우에는 숫자 12(`<eos>` 토큰)이 제거되었고, 디코더의 출력의 경우에는 숫자 11(`<sos>` 토큰)이 제거되었습니다. 이제 패딩을 진행합니다. 패딩을 진행하면 모든 샘플들의 길이가 정해준 길이로 동이라헥 변환됩니다. 여기서는 아까 저장해두었던 가장 긴 샘플의 길이인 `max_eng_seq_len`, `max_fra_seq_len`를 각각 사용하였습니다.

이렇게 되면 영어 데이터의 모든 샘플들은 `max_eng_seq_len`의 길이를 가지고, 프랑스어의 모든 샘플들은 `max_fra_seq_len`의 길이가 되겠죠?

In [21]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')

print("영어 데이터의 크기(shape) :", np.shape(encoder_input))
print("프랑스어 입력 데이터의 크기(shape) :", np.shape(decoder_input))
print("프랑스어 출력 데이터의 크기(shape) :", np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 23)
프랑스어 입력 데이터의 크기(shape) : (50000, 76)
프랑스어 출력 데이터의 크기(shape) : (50000, 76)


모든 샘플들의 길이가 동일하게 변환된 것을 알 수 있습니다. 모든 샘플들의 길이가 동일하게 변환되는 과정에서 정해준 길이보다 짧은 데이터들은 뒤에 0(`<pad>`)으로 채워집니다. 인코더의 샘플 하나만 출력해볼까요?

In [22]:
print(encoder_input[0])

[19  3  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


앞서 [19, 3, 8]이라는 3개의 단어만 있는 샘플이 뒤에 0이 채워지면서 `max_eng_seq_len`의 값인 23의 길이를 가지게 되었습니다. 이제 각 정수에 대해서 벡터화 방법으로 원-핫 인코딩을 선택합니다.

In [23]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

print("영어 데이터의 크기 (shape) :", np.shape(encoder_input))
print("프랑스어 입력데이터의 크기 (shape) :", np.shape(decoder_input))
print("프랑스어 출력데이터의 크기 (shape) :", np.shape(decoder_target))

영어 데이터의 크기 (shape) : (50000, 23, 51)
프랑스어 입력데이터의 크기 (shape) : (50000, 76, 73)
프랑스어 출력데이터의 크기 (shape) : (50000, 76, 73)


원-핫 인코딩을 하고 나서의 데이터의 크기는 (샘플의 수 X 샘플의 길이 X 단어장의 크기)가 됩니다. 원-핫 인코딩은 각 정수를 단어장의 크기를 가지는 원-핫 벡터로 인코딩하는 과정이기 때문입니다.

마지막으로, 훈련 과정의 validation을 위해 위에서 생성한 데이터 50000건 중 3000건만 검증 데이터로 삼고, 나머지를 학습 데이터로 삼겠습니다.

In [24]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print("영어 학습 데이터의 크기 (shape) :", np.shape(encoder_input))
print("프랑스어 학습 입력데이터의 크기 (shape) :", np.shape(decoder_input))
print("프랑스어 학습 출력데이터의 크기 (shape) :", np.shape(decoder_target))

영어 학습 데이터의 크기 (shape) : (50000, 23, 51)
프랑스어 학습 입력데이터의 크기 (shape) : (50000, 76, 73)
프랑스어 학습 출력데이터의 크기 (shape) : (50000, 76, 73)


## 모델 훈련하기

먼저 필요한 도구들을 import 합시다.

In [25]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

print("슝=3")

슝=3


먼저 인코더를 설계해볼까요?

인코더는 디코더보다 상대적으로 간단합니다. LSTM 셸을 설계하고 나서 문장을 입력받으면 LSTM 셸이 마지막으로 time step의 hidden state와 cell state를 전달받아서 저장해줍니다. 앞서 인코더의 마지막 hidden state를 디코더의 첫 번째 Hidden state로 사용한다고 했었죠? 일반적인 기본 RNN의 경우에는 그것이 맞미나, 기본 RNN보다 좀 더 복잡한 LSTM의 경우에는 hidden state 뿐만 아니라 cell state라는 것이 존재합니다.

그래서 인코더 LSTM 셸의 마지막 time step의 **hidden state와 cell state를 같이** 디코더 LSTM의 첫 번째 Hidden state와 cell state로 전달해주어야 합니다.

In [26]:
# 입력 텐서 생성
encoder_inputs = Input(shape=(None, eng_vocab_size))
# hidden state가 256인 인코더의 LSTM 셸을 생성
encoder_lstm = LSTM(units = 256, return_state = True)
# 디코더로 전달할 hidden state, cell state를 리턴, encoder_outputs는 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장
encoder_states = [state_h, state_c]

위 코드를 살펴봅시다.

1. 우선 LSTM의 입력 텐서를 정의해줍니다. 입력 문장을 저장하게 될 변수 텐서입니다.
2. 256의 hidden_size를 가지는 LSTM 셸을 만들어줍니다. LSTM의 *수용력(capacity)*를 의미합니다. <br/> return_state = True로 해서 hidden state와 cell state를 리턴받을 수 있도록 합니다.
3. 입력 텐서를 입력으로 마지막 time step의 hidden state와 cell state를 결과로 받습니다.
4. 마지막 time step의 hidden state와 cell state를 encoder_states라는 하나의 변수에 저장해두었습니다. 이를 디코더에 전달하면 됩니다.

이제 디코더를 설계해봅시다! 디코더도 인코더랑 몇 가지 세부 사항을 제외하고 거의 똑같습니다.

In [27]:
# 입력 텐서 생성
decoder_inputs = Input(shape=(None, fra_vocab_size))
# hidden size가 256인 인코더의 LSTM셸 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state = True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

세 번재 줄을 보면 디코더의 인자로 `initial_state`가 추가되었습니다. LSTM 셸의 초기 상태를 정의해줄 수 있는 인자입니다. 여기서는 이전에 저장한 인코더의 마지막 time step의 hidden state와 cell state를 사용하였습니다. 

디코더의 출력 층을 설계해줍시다.

In [28]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

매 time step마다 다중 클래스 분류 문제이므로 프랑스어 단어장으로부터 한 가지 문자만 선택하도록 합니다. Dense의 인자로 프랑스어 단어장의 크기를 기재하고, 활성화 함수로 소프트맥수 함수를 사용, 최종적으로 인코더와 디코더를 연결해서 하나의 모델로 만들어줍니다. Model의 input과 Output의 정의를 유심히 살펴주세요

In [29]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 51)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 315392      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 lstm[0][1]            

In [30]:
model.fit(
    x=[encoder_input_train, decoder_input_train],
    y = decoder_target_train,
    validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
    batch_size = 128,
    epochs = 50
)

Epoch 1/50
368/368 [==============================] - 48s 130ms/step - loss: 0.9374 - val_loss: 0.8003
Epoch 2/50
368/368 [==============================] - 47s 128ms/step - loss: 0.5709 - val_loss: 0.6403
Epoch 3/50
368/368 [==============================] - 47s 128ms/step - loss: 0.4682 - val_loss: 0.5627
Epoch 4/50
368/368 [==============================] - 48s 130ms/step - loss: 0.4105 - val_loss: 0.5087
Epoch 5/50
368/368 [==============================] - 48s 130ms/step - loss: 0.3726 - val_loss: 0.4803
Epoch 6/50
368/368 [==============================] - 50s 136ms/step - loss: 0.3457 - val_loss: 0.4551
Epoch 7/50
368/368 [==============================] - 51s 137ms/step - loss: 0.3261 - val_loss: 0.4359
Epoch 8/50
368/368 [==============================] - 49s 133ms/step - loss: 0.3096 - val_loss: 0.4203
Epoch 9/50
368/368 [==============================] - 50s 135ms/step - loss: 0.2961 - val_loss: 0.4102
Epoch 10/50
368/368 [==============================] - 50s 135ms/step - l

# 모델 테스트하기

seq2seq는 훈련할때와 테스트 단계의 동작이 다릅니다. 이를 위해서 테스트 단계의 디코더 모델은 설계를 다시해줄 필요가 있습니다. 물론 이전에 학습된 디코더 모델의 레이어는 그대로 사용합니다. 

왜이렇게 번거로운 과정이 생기는 걸까요? Text Generator 모델을 만들어 보신 분이라면 알 수 있습니다. 훈련시에는 학습해야 할 타겟 문장을 디코더 모델의 입력, 출력 시퀀스로 넣어주고 디코더 모델이 타겟 문장을 한꺼번에 출력하게 할 수 있습니다. 그러나 테스트 단계에서는 그럴 수가 없습니다. 하나의 문장을 만들어내기 위해 루프를 돌면서 단어를 하나씩 차례차례 예측해서, 예측한 단어가 다시 다음 단어를 예측할 때 사용되는 입력으로 재사용되는 과정이 진행되기 때문입니다.

정리하면, 테스트 단계에서의 디코더의 동작 순서는 아래와 같습니다.

1. 인코더에 입력 문장을 넣어 마지막 time step의 hidden, cell state를 얻는다
2. 토큰인 '\t'를 디코더에 입력한다.
3. 이전 time step의 출력층의 예측 결과를 현재 time step의 입력으로 ㅏㅎㄴ다.
4. 3을 반복하다가 토큰인 '\n'가 예측되면 이를 중단한다.

이를 구현하기 위해서 훈련 과정에서와의 차이점은 이전 time step의 출력층의 예측 결과를 현재 time step의 입력으로 사용하는 단계를 추가하기 위해서 루프를 돌며 디코더의 LSTM 셸을 마치 수동 제어하는 느낌으로 설계해야 합니다. 이 과정에서 코드가 좀 더 길어지게 됩니다.

우선 인코더를 정의합니다. encoder_inputs와 encoder_states는 이미 정의한 것들을 재사용합니다.

In [31]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 51)]        0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 315392    
Total params: 315,392
Trainable params: 315,392
Non-trainable params: 0
_________________________________________________________________


이제 디코더를 설계합니다.

In [49]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape = (256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape = (256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time_step의 초기 상태로 사용
# 구체적인 동작 자체는 def decode_sequence()에 구현
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states = [state_h, state_c]

훈련 과정에서의 디코더보다 코드가 좀 더 길어졌죠? 이전 time step의 출력 결과를 현재 time step의 입력으로 사용하기 위해서 디코더 LSTM 셸 동작을 좀 더 세분화해서 구현했습니다. 동작 자체는 이후에 구현할 def decode_sequence()에서 좀 더 자세히 다루겠습니다.

디코더의 출력층을 재설계해줍니다.

In [50]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs = [decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 input_6[0][0]         

In [51]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

이제 예측 과정을 위한 함수 `decode_sequence()`를 구현합니다. `decode_sequence()`의 입력으로 들어가는 것은 번역하고자 하는 문장의 정수 시퀀스입니다. `decode_sequence()` 내부에는 인코더를 구현한 encoder_model이 있어서 이 모델에 번역하고자 하는 문장의 정수 시퀀스인 `input_seq`를 입력하면 encoder_model은 마지막 시점의 hidden state를 리턴합니다.

```python
state_value = encoder_model.predict(input_seq)
```

이 hidden state는 디코더의 첫번째 시점의 hidden state가 되고, 디코더는 이제 번역 문장을 완성하기 위한 예측 과정을 진행합니다. 디코더의 예측 과정에서는 이전 시점에서 예측한 단어를 디코더의 현재 시점의 입력으로 넣어주는 작업을 진행합니다. 그리고 이 작업은 종료를 의미하는 종료 토큰을 만나거나, 주어진 최대 길이를 넘을 때까지 반복합니다.

In [52]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)
    
    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""
    
    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이전 시점에서 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]
        
        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char
        
        # <eos>에 도달하거나 최대 길이를 넘으면 중단
        if (sampled_char == '\n') or len(decoded_sentence) > max_fra_seq_len:
            stop_condition = True
        
        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.
        
        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]
        
    
    return decoded_sentence

이렇게 구현한 함수를 임의의 인덱스의 번역하고자 하는 문장 샘플을 입력하여 출력 결과를 테스트해보겠습니다.

In [55]:
import numpy as np

for seq_index in [3, 50, 100, 1001]:
    input_seq = encoder_input[seq_index : seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print("입력 문장 :", lines.eng[seq_index])
    print("정답 문장 :", lines.fra[seq_index][1:len(lines.fra[seq_index]) -1])
    print("번역기가 번약한 문장 :", decoded_sentence[:len(decoded_sentence)-1])

-----------------------------------
입력 문장 : Run!
정답 문장 :  Cours ! 
번역기가 번약한 문장 :  courez ! 
-----------------------------------
입력 문장 : I left.
정답 문장 :  Je suis partie. 
번역기가 번약한 문장 :  je suis partie. 
-----------------------------------
입력 문장 : Call us.
정답 문장 :  Appelez-nous ! 
번역기가 번약한 문장 :  appelle ! 
-----------------------------------
입력 문장 : Turn left.
정답 문장 :  Tourne à gauche. 
번역기가 번약한 문장 :  te lendome-moi. 


일부 정답 문장과 다른 번역을 하는 경우도 있지만, 대체적으로 정답 문장과 거의 비슷한 번역을 확인할 수 있습니다.